In [1]:
#!/usr/bin/env python

def _bitlen(x):
    assert x >= 0
    return len(bin(x)) - 2


def _shift(x, s):
    assert _bitlen(x) <= 16
    return ((x << s) & 0xFFFF) | (x >> (16 - s))


def _mod(x):
    return ((x - 1) % 8) + 1

S7 = (
     54, 50, 62, 56, 22, 34, 94, 96, 38,  6, 63, 93, 2,  18,123, 33,
     55,113, 39,114, 21, 67, 65, 12, 47, 73, 46, 27, 25,111,124, 81,
     53,  9,121, 79, 52, 60, 58, 48,101,127, 40,120,104, 70, 71, 43,
     20,122, 72, 61, 23,109, 13,100, 77,  1, 16,  7, 82, 10,105, 98,
    117,116, 76, 11, 89,106,  0,125,118, 99, 86, 69, 30, 57,126, 87,
    112, 51, 17,  5, 95, 14, 90, 84, 91,  8, 35,103, 32, 97, 28, 66,
    102, 31, 26, 45, 75,  4, 85, 92, 37, 74, 80, 49, 68, 29,115, 44,
     64,107,108, 24,110, 83, 36, 78, 42, 19, 15, 41, 88,119, 59,  3,
)

S9 = (    
    167,239,161,379,391,334,  9,338, 38,226, 48,358,452,385, 90,397,
    183,253,147,331,415,340, 51,362,306,500,262, 82,216,159,356,177,
    175,241,489, 37,206, 17,  0,333, 44,254,378, 58,143,220, 81,400,
     95,  3,315,245, 54,235,218,405,472,264,172,494,371,290,399, 76,
    165,197,395,121,257,480,423,212,240, 28,462,176,406,507,288,223,
    501,407,249,265, 89,186,221,428,164, 74,440,196,458,421,350,163,
    232,158,134,354, 13,250,491,142,191, 69,193,425,152,227,366,135,
    344,300,276,242,437,320,113,278, 11,243, 87,317, 36, 93,496, 27,
    487,446,482, 41, 68,156,457,131,326,403,339, 20, 39,115,442,124,
    475,384,508, 53,112,170,479,151,126,169, 73,268,279,321,168,364,
    363,292, 46,499,393,327,324, 24,456,267,157,460,488,426,309,229,
    439,506,208,271,349,401,434,236, 16,209,359, 52, 56,120,199,277,
    465,416,252,287,246,  6, 83,305,420,345,153,502, 65, 61,244,282,
    173,222,418, 67,386,368,261,101,476,291,195,430, 49, 79,166,330,
    280,383,373,128,382,408,155,495,367,388,274,107,459,417, 62,454,
    132,225,203,316,234, 14,301, 91,503,286,424,211,347,307,140,374,
     35,103,125,427, 19,214,453,146,498,314,444,230,256,329,198,285,
     50,116, 78,410, 10,205,510,171,231, 45,139,467, 29, 86,505, 32,
     72, 26,342,150,313,490,431,238,411,325,149,473, 40,119,174,355,
    185,233,389, 71,448,273,372, 55,110,178,322, 12,469,392,369,190,
      1,109,375,137,181, 88, 75,308,260,484, 98,272,370,275,412,111,
    336,318,  4,504,492,259,304, 77,337,435, 21,357,303,332,483, 18,
     47, 85, 25,497,474,289,100,269,296,478,270,106, 31,104,433, 84,
    414,486,394, 96, 99,154,511,148,413,361,409,255,162,215,302,201,
    266,351,343,144,441,365,108,298,251, 34,182,509,138,210,335,133,
    311,352,328,141,396,346,123,319,450,281,429,228,443,481, 92,404,
    485,422,248,297, 23,213,130,466, 22,217,283, 70,294,360,419,127,
    312,377,  7,468,194,  2,117,295,463,258,224,447,247,187, 80,398,
    284,353,105,390,299,471,470,184, 57,200,348, 63,204,188, 33,451,
     97, 30,310,219, 94,160,129,493, 64,179,263,102,189,207,114,402,
    438,477,387,122,192, 42,381,  5,145,118,180,449,293,323,136,380,
     43, 66, 60,455,341,445,202,432,  8,237, 15,376,436,464, 59,461,
)


class Kasumi:
    def __init__(self):
        self.key_KL1 = [None] * 9
        self.key_KL2 = [None] * 9
        self.key_KO1 = [None] * 9
        self.key_KO2 = [None] * 9
        self.key_KO3 = [None] * 9
        self.key_KI1 = [None] * 9
        self.key_KI2 = [None] * 9
        self.key_KI3 = [None] * 9


    def set_key(self, master_key):
        assert _bitlen(master_key) <= 128

        key       = [None] * 9
        key_prime = [None] * 9

        master_key_prime = master_key ^ 0x0123456789ABCDEFFEDCBA9876543210
        for i in range(1, 9):
            key[i]       = (master_key       >> (16 * (8 - i))) & 0xFFFF
            key_prime[i] = (master_key_prime >> (16 * (8 - i))) & 0xFFFF

        for i in range(1, 9):
            self.key_KL1[i] = _shift(key[_mod(i + 0)], 1)
            self.key_KL2[i] =  key_prime[_mod(i + 2)]
            self.key_KO1[i] = _shift(key[_mod(i + 1)], 5)
            self.key_KO2[i] = _shift(key[_mod(i + 5)], 8)
            self.key_KO3[i] = _shift(key[_mod(i + 6)], 13)
            self.key_KI1[i] =  key_prime[_mod(i + 4)]
            self.key_KI2[i] =  key_prime[_mod(i + 3)]
            self.key_KI3[i] =  key_prime[_mod(i + 7)]


    def fun_FI(self, input, round_key):
        # assert _bitlen(input)  <= 16
        
        left  = input >> 7
        right = input & 0b1111111

        round_key_1 = round_key >> 9
        round_key_2 = round_key & 0b111111111

        tmp_l = right
        # assert _bitlen(left)  <= 9
        tmp_r = S9[left] ^ right

        left  = tmp_r ^ round_key_2
        # assert _bitlen(tmp_l) <= 7
        right = S7[tmp_l] ^ (tmp_r & 0b1111111) ^ round_key_1

        tmp_l = right
        # assert _bitlen(left)  <= 9
        tmp_r = S9[left] ^ right

        # assert _bitlen(tmp_l) <= 7
        left  = S7[tmp_l] ^ (tmp_r & 0b1111111)
        right = tmp_r

        # assert _bitlen(left)  <= 7
        # assert _bitlen(right) <= 9
        return (left << 9) | right


    def fun_FO(self, input, round_i):
        # assert _bitlen(input)  <= 32
        # assert round_i >= 1 and round_i <= 8

        in_left  = input >> 16
        in_right = input & 0xFFFF

        out_left  = in_right # this is not Feistel at all, maybe not reversible
        out_right = self.fun_FI(in_left ^ self.key_KO1[round_i], 
                                          self.key_KI1[round_i]) ^ in_right

        in_left   = out_right # use in_* as temp variables
        in_right  = self.fun_FI(out_left ^ self.key_KO2[round_i],
                                           self.key_KI2[round_i]) ^ out_right

        out_left  = in_right
        out_right = self.fun_FI(in_left ^ self.key_KO3[round_i], 
                                          self.key_KI3[round_i]) ^ in_right

        # assert _bitlen(out_left)  <= 16
        # assert _bitlen(out_right) <= 16
        return (out_left << 16) | out_right


    def fun_FL(self, input, round_i):
        # assert _bitlen(input)  <= 32
        # assert round_i >= 1 and round_i <= 8

        in_left  = input >> 16
        in_right = input & 0xFFFF

        out_right = in_right ^ _shift(in_left   & self.key_KL1[round_i], 1)
        out_left  = in_left  ^ _shift(out_right | self.key_KL2[round_i], 1)

        # assert _bitlen(out_left)  <= 16
        # assert _bitlen(out_right) <= 16
        return (out_left << 16) | out_right


    def fun_f(self, input, round_i):
        # assert _bitlen(input)  <= 32
        # assert round_i >= 1 and round_i <= 8

        if round_i % 2 == 1:
            state  = self.fun_FL(input, round_i)
            output = self.fun_FO(state, round_i)
        else:
            state  = self.fun_FO(input, round_i)
            output = self.fun_FL(state, round_i)

        # assert _bitlen(output) <= 32
        return output


    def enc_1r(self, in_left, in_right, round_i):
        # assert _bitlen(in_left)  <= 32
        # assert _bitlen(in_right) <= 32
        # assert round_i >= 1 and round_i <= 8

        out_right = in_left # note this is different from normal Feistel
        out_left  = in_right ^ self.fun_f(in_left, round_i)

        # assert _bitlen(out_left)  <= 32
        # assert _bitlen(out_right) <= 32
        return out_left, out_right


    def dec_1r(self, in_left, in_right, round_i):
        # assert _bitlen(in_left)  <= 32
        # assert _bitlen(in_right) <= 32
        # assert round_i >= 1 and round_i <= 8

        out_left  = in_right
        out_right = self.fun_f(in_right, round_i) ^ in_left

        # assert _bitlen(out_left)  <= 32
        # assert _bitlen(out_right) <= 32
        return out_left, out_right


    def enc(self, plaintext):
        assert _bitlen(plaintext) <= 64
        left  = plaintext >> 32
        right = plaintext & 0xFFFFFFFF
        for i in range(1, 9):
            left, right = self.enc_1r(left, right, i)
        return (left << 32) | right


    def dec(self, ciphertext):
        assert _bitlen(ciphertext) <= 64
        left  = ciphertext >> 32
        right = ciphertext & 0xFFFFFFFF
        for i in range(8, 0, -1):
            left, right = self.dec_1r(left, right, i)
        return (left << 32) | right


if __name__ == '__main__':
    key     = 0x9900aabbccddeeff1122334455667788    
    text    = 0xfedcba0987654321

    print ('Text is:   ', hex(text))
    print ('\n')
    
    my_kasumi = Kasumi()
    my_kasumi.set_key(key)

    encrypted = my_kasumi.enc(text)
    print ('Encrypted: ', hex(encrypted))

    for i in range(99): # for testing
        encrypted = my_kasumi.enc(encrypted)
    for i in range(99):
        encrypted = my_kasumi.dec(encrypted)

    decrypted = my_kasumi.dec(encrypted)
    print ('Decrypted: ', hex(decrypted))
   

Text is:    0xfedcba0987654321


Encrypted:  0x514896226caa4f20
Decrypted:  0xfedcba0987654321


In [3]:
import random

def append_hex(a, b):
    sizeof_b = 0

    # get size of b in bits
    while((b >> sizeof_b) > 0):
        sizeof_b += 1

    # align answer to nearest 4 bits (hex digit)
    sizeof_b += sizeof_b % 4

    return (a << sizeof_b) | b


key_a        = 0x9900aabbccddeeff1122334455667788
delta_key_ab = 0x00008000000000000000000000000000
delta_key_ac = 0x00000000000000000000000080000000
key_b = key_a ^ delta_key_ab
key_c = key_a ^ delta_key_ac
key_d = key_c ^ delta_key_ab

delta = 0x0000000000100000
nabla = 0x0010000000000000

C_a = [None] * 2**24   #2**24
P_a = [None] * 2**24   #2**24
P_b = [None] * 2**24   #2**24
C_b = [None] * 2**24   #2**24
C_c = [None] * 2**24   #2**24
P_c = [None] * 2**24   #2**24
P_d = [None] * 2**24   #2**24
C_d = [None] * 2**24   #2**24

A = 0x2950412b
Kasumi_P_a = Kasumi()
Kasumi_P_a.set_key(key_a)
Kasumi_P_b = Kasumi()
Kasumi_P_b.set_key(key_b)
Kasumi_P_c = Kasumi()
Kasumi_P_c.set_key(key_c)
Kasumi_P_d = Kasumi()
Kasumi_P_d.set_key(key_d)

dict = {}
quartets = {}

for i in range(0, 2**24):  #2**24
    #print("****************")
    X_a = random.getrandbits(32)
    C_a[i] = hex(append_hex(X_a, A)) #f'0x{A:x}{X_a:x}'
    #print(C_a[i])
    
    P_a[i] = hex(Kasumi_P_a.dec(int(C_a[i], base = 16)))
    #print(P_a[i])
    
    P_b[i] = int(P_a[i], base = 16) ^ delta
    C_b[i] = Kasumi_P_b.enc(P_b[i])
    #print(hex(P_b[i]))
    #print(hex(C_b[i]))
    
    #stavljam u hes tabelu
    dict[hex(C_b[i] & 0xFFFFFFFF)] = (C_a[i], hex(C_b[i])) 
    
    
for i in range(0, 2**24):  
    #print("-------------------")
    Y_c = random.getrandbits(32)
    C_c[i] = hex(append_hex(Y_c, A ^ 0x00100000))
    print(C_c[i])
    
    P_c[i] = hex(Kasumi_P_c.dec(int(C_c[i], base = 16)))
    #print(P_c[i])
    
    P_d[i] = int(P_c[i], base = 16) ^ delta
    C_d[i] = Kasumi_P_d.enc(P_d[i])
    #print(hex(P_d[i]))
    #print(hex(C_d[i]))
        
    if hex((C_d[i] & 0xFFFFFFFF) ^ nabla) in dict:
        quartets[hex(int(hex(C_a)[:10], base = 16) ^ int(hex(C_c)[:10], base = 16))] = \
            ( dict[hex((C_d[i] & 0xFFFFFFFF) ^ nabla)][0], \
              dict[hex((C_d[i] & 0xFFFFFFFF) ^ nabla)][1], \
              hex(C_c), \
              hex(C_d)\
            )
    
    
        
print(quartets)     

****************
0xbca7e8242950412b
0x24af8968213f8d1e
0x24af8968212f8d1e
0xb0bc95ce1bda542e
****************
0x812d8a722950412b
0xd7548dc8e63542b
0xd7548dc8e73542b
0x70feca8882a0a8a7
****************
0xba722eeb2950412b
0xb7d269a8b3c7c6be
0xb7d269a8b3d7c6be
0x2fb12d0781e8c3e2
****************
0x2329c2ee2950412b
0x99a5d06e52f8ccab
0x99a5d06e52e8ccab
0xc420d3356bc416e
****************
0x20aba6582950412b
0x970b40d9d3f7fa05
0x970b40d9d3e7fa05
0xc19ba193f1ccceef
****************
0xfe5f20d72950412b
0x25cd96a0922744c3
0x25cd96a0923744c3
0xc6b1a1ea948074a8
****************
0x48a888632950412b
0xb669b2ae141d2621
0xb669b2ae140d2621
0xb4d6319a619c34ce
****************
0xe565bfc52950412b
0xfc30606b592461fb
0xfc30606b593461fb
0x2e1a3e9563e6c1d4
****************
0xa46f04382950412b
0xdedefce72229fa24
0xdedefce72239fa24
0x94123a1ec0287ae6
****************
0x6e85349c2950412b
0x6409522eda4f2452
0x6409522eda5f2452
0x8c081b13716fae94
****************
0xdfc22d4e2950412b
0xf5344182489a6d3c
0xf5344182488a6d3c